**About** : This notebook performs inference on the test data. It is used to generate pseudo-labels.
  - Use the `log_folder` parameter to specify the experiment.
  - Use the `use_tta` parameter to speciy whether to use test time augmentations.
  - Use the `save` parameter to save predictions.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import json
import torch
import warnings
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

sys.path.append("../code/")
# warnings.simplefilter("ignore", UserWarning)

In [ ]:
from params import *
from inference.main_test import k_fold_inf_test

### Inference

In [ ]:
log_folder = "../logs/2021-05-01/2/"  # b1 512 2 fix
# log_folder = "../logs/2021-05-06/1/"  # b1 512 2 fix debug

In [ ]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [ ]:
config.overlap_factor = 1.5
use_tta = False
global_threshold = 0.5
config.selected_folds = [0, 1, 2, 3, 4]
save = False

In [ ]:
df = pd.read_csv(DATA_PATH + "sample_submission.csv")
images = df['id'].values.tolist() + ["VAN0003-LK-32-21-PAS_registered.ome", "VAN0011-RK-3-10-PAS_registered.ome"]

In [ ]:
%%time
scores = k_fold_inf_test(
    config,
    images,
    log_folder=log_folder,
    use_full_size=False,
    global_threshold=global_threshold,
    use_tta=use_tta,
    save=save,
)